# Tip Prediction, Part I: Data Cleanup & Feature Creation

Author's Workday ID: C175799, Initials: RPR

## Choose GPU

In [6]:
# From: https://github.com/keras-team/keras/issues/6031
import os
gpu_id = input( "Select GPU [0 or 1]: " )

if gpu_id in [ "0", "1" ]:
    os.environ[ "CUDA_VISIBLE_DEVICES" ] = gpu_id
else:
    print( "Invalid GPU id.  Defaulting to '0,1'" )

Select GPU [0 or 1]: 1


## Imports and Util Functions

In [7]:
import pandas as pd
import numpy as np
import time
import datetime
import math
import random

def get_time( output=True ):
    
    temp = time.time()
    if output:
        now = datetime.datetime.now()
        print( now.strftime( "%Y.%m.%d %H:%M" ) )
        
    return temp

foo = get_time()

def print_time( start_time, end_time, interval="seconds" ):
    
    if interval == "hours":
        print ( "Time to process: [%s] hours" % ( str( ( end_time - start_time ) / 60 / 60 ) ) )
    else:
        print ( "Time to process: [%s] seconds" % ( str( end_time - start_time ) ) )

print_time( 0, 3600, interval="hours" )

verbose = False

2018.06.01 15:32
Time to process: [1.0] hours


## Load Data & Rename Columns

In [8]:
page_start = get_time()
trips = pd.read_csv( "data/green-tripdata-2015-09-raw.csv" )
trips.columns = map( str.lower, trips.columns )
if verbose: print( list( trips.columns ) )
print_time( page_start, get_time() )

2018.06.01 15:32
2018.06.01 15:32
Time to process: [2.868013620376587] seconds


## Quick Look: What Does Data Look Like?

In [9]:
rows_original = trips.shape[ 0 ]
print( "Rows in dataframe: ", rows_original )

# eyeball unruly values
trips.describe()

Rows in dataframe:  1494926


,vendorid,ratecodeid,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type
count,1.494926e+06,1.494926e+06,1.494926e+06,1.494926e+06,1.494926e+06,1.494926e+06,1.494926e+06,1.494926e+06,1.494926e+06,1.494926e+06,1.494926e+06,1.494926e+06,1.494926e+06,0.0,1.494926e+06,1.494926e+06,1.494926e+06,1.494922e+06
mean,1.782045e+00,1.097653e+00,-7.383084e+01,4.069114e+01,-7.383728e+01,4.069291e+01,1.370598e+00,2.968141e+00,1.254320e+01,3.512800e-01,4.866408e-01,1.235727e+00,1.231047e-01,NaN,2.920991e-01,1.503215e+01,1.540559e+00,1.022353e+00
std,4.128570e-01,6.359437e-01,2.776082e+00,1.530882e+00,2.677911e+00,1.476698e+00,1.039426e+00,3.076621e+00,1.008278e+01,3.663096e-01,8.504473e-02,2.431476e+00,8.910137e-01,NaN,5.074009e-02,1.155316e+01,5.232935e-01,1.478288e-01
min,1.000000e+00,1.000000e+00,-8.331908e+01,0.000000e+00,-8.342784e+01,0.000000e+00,0.000000e+00,0.000000e+00,-4.750000e+02,-1.000000e+00,-5.000000e-01,-5.000000e+01,-1.529000e+01,NaN,-3.000000e-01,-4.750000e+02,1.000000e+00,1.000000e+00
25%,2.000000e+00,1.000000e+00,-7.395961e+01,4.069895e+01,-7.396782e+01,4.069878e+01,1.000000e+00,1.100000e+00,6.500000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,NaN,3.000000e-01,8.160000e+00,1.000000e+00,1.000000e+00
50%,2.000000e+00,1.000000e+00,-7.394536e+01,4.074674e+01,-7.394504e+01,4.074728e+01,1.000000e+00,1.980000e+00,9.500000e+00,5.000000e-01,5.000000e-01,0.000000e+00,0.000000e+00,NaN,3.000000e-01,1.176000e+01,2.000000e+00,1.000000e+00
75%,2.000000e+00,1.000000e+00,-7.391748e+01,4.080255e+01,-7.391013e+01,4.079015e+01,1.000000e+00,3.740000e+00,1.550000e+01,5.000000e-01,5.000000e-01,2.000000e+00,0.000000e+00,NaN,3.000000e-01,1.830000e+01,2.000000e+00,1.000000e+00
max,2.000000e+00,9.900000e+01,0.000000e+00,4.317726e+01,0.000000e+00,4.279934e+01,9.000000e+00,6.031000e+02,5.805000e+02,1.200000e+01,5.000000e-01,3.000000e+02,9.575000e+01,NaN,3.000000e-01,5.813000e+02,5.000000e+00,2.000000e+00


## Observation

At a glance, we see that: 

1. ehail_fee is full of NaNs, so we'll drop this column
2. fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount all have negative and/or zero values. Not known: whether neg numbers are refunds or errors, so we'll drop them.
3. passenger_count has 0 as a min
4. pickup/dropoff lat/lons have 0's for mins/max's, which is inappropriate for the given geographic context.  We'll address in a naive way towards the end

## Data Cleanup

### Define Min/Max Function

In [10]:
# we'll use this later to check results of cleanup
def display_min_max():

    for column in trips.columns:
        print( column, "min:", trips[ column ].min(), "--- max:", trips[ column ].max() )
        
if verbose: print( display_min_max() )

## Drop

In [11]:
# drop ehail_fee, full of NaNs!
trips.drop( columns=[ "ehail_fee" ], inplace=True )

## Filter Counterintuitive Values

In [12]:
# drop records w/ counterintuitively negative values
trips = trips[ trips.fare_amount > 0 ]
trips = trips[ trips.extra >= 0 ]
trips = trips[ trips.mta_tax >= 0 ]
trips = trips[ trips.tip_amount >= 0 ]
trips = trips[ trips.tolls_amount >= 0 ]
trips = trips[ trips.improvement_surcharge >= 0 ]
# http://www.nyc.gov/html/tlc/html/passenger/taxicab_rate.shtml
trips = trips[ trips.total_amount > 2.5 ]
trips = trips[ trips.passenger_count > 0 ]

# http://www.nyc.gov/html/tlc/downloads/pdf/data_dictionary_trip_records_green.pdf
trips = trips[ trips.ratecodeid <= 6 ]

if verbose: print( "Records dropped with negative, and/or counter-intuitive values:", rows_original - len( trips ) )

## Feature Generation: Pickup & Dropoff

In [13]:
# convert to datetime
trips.lpep_dropoff_datetime = pd.to_datetime( trips.lpep_dropoff_datetime )
trips.lpep_pickup_datetime = pd.to_datetime( trips.lpep_pickup_datetime )
if verbose: print( trips.lpep_dropoff_datetime.describe() )

In [14]:
trips[ 'trip_duration_mins' ] = ( 

    ( trips.lpep_dropoff_datetime - trips.lpep_pickup_datetime ).apply( 
    lambda duration: duration.total_seconds() / 60.0 ) 
)
if verbose: print( trips[ 'trip_duration_mins' ].describe() )

In [15]:
trips[ "pickup_day_of_week" ] = trips.lpep_pickup_datetime.dt.dayofweek
if verbose: print( trips[ "pickup_day_of_week" ].describe() )

In [16]:
trips[ "pickup_day" ] = trips.lpep_pickup_datetime.dt.day
if verbose: print( trips[ "pickup_day" ].describe() )

In [17]:
trips[ "pickup_hour" ] = trips.lpep_pickup_datetime.dt.hour
if verbose: print( trips[ "pickup_hour" ].describe() )

In [18]:
trips[ "pickup_minute" ] = trips.lpep_pickup_datetime.dt.minute
if verbose: print( trips[ "pickup_minute" ].describe() )

In [19]:
trips[ "dropoff_day_of_week" ] = trips.lpep_dropoff_datetime.dt.dayofweek
if verbose: print( trips[ "dropoff_day_of_week" ].describe() )

In [20]:
trips[ "dropoff_day" ] = trips.lpep_pickup_datetime.dt.day
if verbose: print( trips[ "dropoff_day" ].describe() )

In [21]:
trips[ "dropoff_hour" ] = trips.lpep_dropoff_datetime.dt.hour
if verbose: print( trips[ "dropoff_hour" ].describe() )

In [22]:
trips[ "dropoff_minute" ] = trips.lpep_dropoff_datetime.dt.minute
if verbose: print( trips[ "dropoff_minute" ].describe() )

In [23]:
# drop pickup/dropoff?
trips.drop( columns=[ "lpep_dropoff_datetime", "lpep_pickup_datetime" ], inplace=True )

## Filter Counterintuitive Lat/Lons for Pickup & Dropoff

In [24]:
# Dropoffs
if verbose: print( trips.dropoff_longitude.describe() )
if verbose: print( len( trips[ trips.dropoff_longitude == 0.0 ] ) )
trips = trips[ trips.dropoff_longitude < 0.0 ]
if verbose: print( len( trips[ trips.dropoff_longitude == 0.0 ] ) )
if verbose: print( trips.dropoff_longitude.describe() )
    
# Pickups
if verbose: print( trips.pickup_longitude.describe() )
if verbose: print( len( trips[ trips.pickup_longitude == 0.0 ] ) )
trips = trips[ trips.pickup_longitude < 0.0 ]
if verbose: print( len( trips[ trips.pickup_longitude == 0.0 ] ) )
if verbose: print( trips.pickup_longitude.describe() )

In [25]:
# simply dropping 0 values for longitudes also cleaned up zero'd latitudes too!
if verbose: display_min_max()

## Calculate Tips as Percentage

In [26]:
# add tip as percent
trips[ "tip_percent" ] = ( trips.tip_amount / trips.fare_amount ) * 100
# simply dropping 0 values for longitudes also cleaned up zero'd latitudes too!
if verbose: print( trips.tip_percent.describe() )

In [27]:
# who's the *HUGE* tipper?
max_tip = trips.tip_percent.max()
if verbose: print( max_tip )
trips[ trips.tip_percent == max_tip ]
# ...oh, nevermind :-|

,vendorid,store_and_fwd_flag,ratecodeid,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,trip_distance,fare_amount,...,trip_duration_mins,pickup_day_of_week,pickup_day,pickup_hour,pickup_minute,dropoff_day_of_week,dropoff_day,dropoff_hour,dropoff_minute,tip_percent
88505,2,N,5,-73.926178,40.672138,-73.92617,40.672153,1,0.0,0.1,...,0.05,3,3,3,26,3,3,3,26,15000.0


In [28]:
# remove tip outliers 
tip_p_sd   = trips.tip_percent.std()
tip_p_mean = trips.tip_percent.mean()

# limit tips to w/in 3 stdv
if verbose: print( len( trips[ trips.tip_percent > tip_p_mean + 3 * tip_p_sd ] ) )
trips = trips[ trips.tip_percent <= tip_p_mean + 3 * tip_p_sd ]
if verbose: print( len( trips[ trips.tip_percent > tip_p_mean + 3 * tip_p_sd ] ) )

## ¿Convert Store and Forward?

In [29]:
# convert char flag to boolean
if verbose: print( trips.store_and_fwd_flag.unique() )
y_n_boolean_dict = { 'Y':True, 'N':False }
trips.store_and_fwd_flag = trips.store_and_fwd_flag.map( y_n_boolean_dict )
if verbose: print( trips.store_and_fwd_flag.unique() )

## Calculate Ground Speed

In [30]:
# calc ground speed
trips[ "speed_mph" ] = trips.trip_distance / trips.trip_duration_mins * 60
if verbose: print( trips.speed_mph.describe() )

# remove ridiculous outliers
# 1st, limit to less than 100mph
## Calculate Ground Speedprint( len( trips ) )
trips = trips[ trips.speed_mph < 100 ]
## Calculate Ground Speedprint( len( trips ) )

# 2nd: limit to 3 stdv
mph_mean = trips.speed_mph.mean()
mph_sd   = trips.speed_mph.std()

# limit mph to w/in 3 stdv
if verbose: print( len( trips[ trips.speed_mph > mph_mean + 3 * mph_sd ] ) )
trips = trips[ trips.speed_mph <= mph_mean + 3 * mph_sd ]
if verbose: print( len( trips[ trips.speed_mph > mph_mean + 3 * mph_sd ] ) )

if verbose: print( trips.speed_mph.describe() )

In [31]:
# After bounding trip durations, are trip distances reasonable?
if verbose: print( trips.trip_distance.describe() )

## Payment Types

In [32]:
# take a look at payment types: exploratory work in R showed that payment type affects *recorded* 
# tips: credit card tips avg ~20%, cash ~0%
trips[ "credit_card" ] = trips.payment_type == 1
if verbose: print( trips.credit_card.describe() )

# tips: credit card tips avg ~20%, cash ~0%
trips[ "cash" ] = trips.payment_type == 2
if verbose: print( trips.cash.describe() )
    
# add flag for whether they'll tip or not, based on observation above
trips[ "tip_recorded" ] = trips.tip_amount > 0
if verbose: print( trips.tip_recorded.describe() )

## Calculate Trip Bearing

In [33]:
# From: https://gist.github.com/jeromer/2005586
def calculate_initial_compass_bearing( pointA, pointB ):
    """
    Calculates the bearing between two points.
    The formulae used is the following:
        θ = atan2(sin(Δlong).cos(lat2),
                  cos(lat1).sin(lat2) − sin(lat1).cos(lat2).cos(Δlong))
    :Parameters:
      - `pointA: The tuple representing the latitude/longitude for the
        first point. Latitude and longitude must be in decimal degrees
      - `pointB: The tuple representing the latitude/longitude for the
        second point. Latitude and longitude must be in decimal degrees
    :Returns:
      The bearing in degrees
    :Returns Type:
      float
    """
    if (type(pointA) != tuple) or (type(pointB) != tuple):
        raise TypeError("Only tuples are supported as arguments")

    lat1 = math.radians(pointA[0])
    lat2 = math.radians(pointB[0])

    diffLong = math.radians(pointB[1] - pointA[1])

    x = math.sin(diffLong) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1)
            * math.cos(lat2) * math.cos(diffLong))

    initial_bearing = math.atan2(x, y)

    # Now we have the initial bearing but math.atan2 return values
    # from -180° to + 180° which is not what we want for a compass bearing
    # The solution is to normalize the initial bearing as shown below
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360

    return compass_bearing

## Test Bearings

In [34]:
# get bearing between dc and nyc: https://www.distancefromto.net/distance-from-washington-d-c-to-new-york
dc = ( 38.90719, -77.03687 )
nyc = ( 40.71278, -74.00594 )
print( calculate_initial_compass_bearing( dc, nyc ) )
print( calculate_initial_compass_bearing( nyc, dc ) )

51.24333015092952
233.18437158882222


## Create Wrapper for Calculating Bearing from Pickup to Dropoff Lat/Lons

In [35]:
def get_bearing( row ):
    
    pickup = ( row.pickup_latitude, row.pickup_longitude )
    dropoff = ( row.dropoff_latitude, row.dropoff_longitude )
    
    return calculate_initial_compass_bearing( pickup, dropoff )

In [36]:
start_time = get_time()
trips[ "pickup_bearing" ] = trips.apply( get_bearing, axis=1 )
print_time( start_time, get_time()  )

2018.06.01 15:32
2018.06.01 15:33
Time to process: [44.559367179870605] seconds


## Calculate This Taxi's Trip Speed w/ Other Taxis in This Day/Hour

### First Calculate Mean MPG per Day/Hour

In [37]:
# build lookup table for mph by day and hour, takes advantage of Panda's indexing of group_by indexing.
trips_mph_day_n_hour_grp = trips.groupby( [ "pickup_day_of_week", "pickup_hour" ] )
trips_mph_day_n_hour = trips_mph_day_n_hour_grp[ "speed_mph" ].mean()

def get_mph( day, hour ):
    
    return trips_mph_day_n_hour[ day ][ hour ]

now = datetime.datetime.now()
get_mph( now.weekday(), now.hour )

10.258393447357737

### Add Peers MPH Means Field

In [38]:
def get_mph_wrapper( row ):
    
    return get_mph( ( row.pickup_day_of_week ), ( row.pickup_hour ) )

In [39]:
start_time = get_time()
trips[ "peer_mean_mph" ] = trips.apply( get_mph_wrapper, axis=1 )
print_time( start_time, get_time()  )

2018.06.01 15:33
2018.06.01 15:37
Time to process: [225.16400384902954] seconds


In [40]:
start_time = get_time()
trips[ "peer_mph_delta" ] = trips.speed_mph - trips.peer_mean_mph
print_time( start_time, get_time()  )

2018.06.01 15:37
2018.06.01 15:37
Time to process: [0.0034067630767822266] seconds


In [41]:
#trips.head()

## Create Lat/Lon, MPH and Tip Bins

In [42]:
# 2 points of lat/lon precision ~ 3/4 mile bins
trips[ "pickup_latitude_bin_2" ] = round( trips.pickup_latitude, 2 )
trips[ "pickup_longitude_bin_2" ] = round( trips.pickup_longitude, 2 )

trips[ "dropoff_latitude_bin_2" ] = round( trips.dropoff_latitude, 2 )
trips[ "dropoff_longitude_bin_2" ] = round( trips.dropoff_longitude, 2 )

trips[ "tip_percent_bin" ] = round( trips.tip_percent, 0 )
trips[ "speed_mph_bin" ] = round( trips.speed_mph, 0 )
trips[ "trip_distance_bin" ] = round( trips.trip_distance, 0 )

# create wrapper
rows = trips.shape[ 0 ]

In [43]:
# create a distance frequency lookup table
distance_summary = trips.trip_distance_bin.value_counts()#.reset_index()
#distance_summary.sort_values( by=[ 'index' ], inplace=True )
#print( distance_summary )

def get_distance_frequency( row ):
    
    return distance_summary[ ( row.trip_distance_bin ) ] / rows
 

In [44]:
start_time = get_time()
trips[ "distance_frequency" ] = trips.apply( get_distance_frequency, axis=1 )
print_time( start_time, get_time() )

2018.06.01 15:37
2018.06.01 15:37
Time to process: [32.95541477203369] seconds


In [45]:
# create a mph frequency lookup table
mph_summary = trips.speed_mph_bin.value_counts()#.reset_index()
# mph_summary.sort_values( by=[ 'index'], inplace=True )
# print( mph_summary )

# create wrapper
rows = trips.shape[ 0 ]

def get_mph_frequency( row ):
    
    return mph_summary[ ( row.speed_mph_bin ) ] / rows

In [46]:
start_time = get_time()
trips[ "mph_frequency" ] = trips.apply( get_mph_frequency, axis=1 )
print_time( start_time, get_time() )

2018.06.01 15:37
2018.06.01 15:38
Time to process: [33.216843366622925] seconds


In [47]:
# create a frequency lookup table
tip_summary = trips.tip_percent_bin.value_counts()#.reset_index()
# tip_summary.sort_values( by=[ 'index'], inplace=True )
tip_summary[ 0 ]

869517

In [48]:
# Pandas provides quick plots
trips.tip_percent_bin.plot.hist( bins=50, figsize=( 10, 6 ), grid=True )

In [49]:
# create wrapper
rows = trips.shape[ 0 ]

def get_tip_frequency( row ):
    
    return tip_summary[ ( row.tip_percent_bin ) ] / rows

rows

1455831

## Write Intermediate Results w/o Mean Encoding to CSV

In [56]:
start_time = get_time()
trips.to_csv( "data/green-tripdata-2015-09-cleaned.csv", compression="gzip", index=False )
print_time( start_time, get_time() )

2018.06.01 17:03
2018.06.01 17:05
Time to process: [68.31705808639526] seconds


## Time to Process Basic Cleanup & Features

In [51]:
print_time( page_start, get_time() )

2018.06.01 15:38
Time to process: [381.4318516254425] seconds
